TITLE: ProDy_autoanaly.ipynb

PURPOSE: To provide an automatic analysis of protein dynamics for an NMR or trajectory file.

PROJECT: Generic.

AUTHOR: Jacob Lloyd North

INSTITUTION: Oregon State University

PRECONDITIONS: Trajectory or NMR file is provided.

INPUTS: 

OUTPUTS: 

SECTION 1: Include commonly used libraries

In [ ]:
# Maths and visualization libraries
import numpy as np          # NumPy
import scipy        # Import all of SciPy
import networkx 
# import pathpy2
import matplotlib.pyplot as plt     # Matplotlib
import umap

In [ ]:
%matplotlib inline

In [ ]:
# Machine learning libraries
# import sklearn
# import torch             # Import all of PyTorch
# import fastai            # Import all of FastAi
# import pydbm            # boltzmann machines

In [ ]:
# BIOLOGY-SPECIFIC LIBRARIES
# Bioinformatics
import biopandas
import Bio          # Biopython
# import pdbtools     # Useful for dealing with pdbs
import dlpdb        # Useful for parsing PDB
# import proteincsm   # Symmetry calculation
# import protein        # Utils for UniProt
# import cathpy
# import pydpi
# import isambard
import pyuniprot
# import elaspic
# import aesop
# import backmap

# import openproteindesign

# import tssv
# import propka
# import bio-pyvol
# import discere

import ssbio

# Structural Biology
import RamachanDraw
import biographs
# import biskit       # BUILD FAILS
# import aleph        # Molecular replacement library
from ensemblator.ensemblator_core import analyze, prepare_input     # Clark, Brereton, Karplus
# import paramagpy      # NMR paramagnetism
# import povme          # Measure volume of pockets in a protein structure
# import gmx-clusterByFeatures

# Molecular Dynamics
# import gromacs

# Kinetics
# import pybindingcurve

# import ContactVis

# import usum

# MD analysis
import mdtraj           # Import all of MDTraj
import MDAnalysis
# import pychimera

# Protein Dynamics
import prody as pd            # Protein dynamics
# import pydtmc           # discrete-time markov chains
# Normal modes of motion
# import pydmd            # Dynamic mode decomposition
# import pynamical        # Dynamical systems 
import pyemma
# import molpx
import msmtools
# import pysfd
# import ipymol
# import pypcazip

In [ ]:
# PHYSICAL CHEMISTRY LIBRARIES

# Quantum chemistry libraries
# import quantum_dynamics
# import qutip

# Statistical thermodynamics
# import curp         # energy (heat) flow analysis -- ONLY in Python2 currently!

In [ ]:
# Cellular biology
# import pysces       # Will copy stuff to Pysces directory for model!

In [ ]:
# UTILITY LIBRARIES
import wget         # to download pdb files

In [ ]:
# DEBUG
# import mdbenchmark  # For optimizing core usage in low-resources machines

SECTION 2: MAIN

In [ ]:
# Get PDB

# PDB_id = input("Please enter a PDB ID:")      # if from the RCSB
pro_name = input("Enter a name label:")         # if a local PDB file
local_pdb_file = 'md_0_1_trajectory_nowater.pdb'

# Print Ramachandran plot of the protein
from RamachanDraw import fetch, phi_psi, plot   

# Draw the Ramachandran plot
# plot(fetch(PDB_id))       # if from the RCSB
plot(local_pdb_file)   # if a local PDB file

# Generating a dictionary to store the phi and psi angles, also return the ignored AA
# phi_psi_dict, ignored_res = phi_psi(fetch(PDB_id), return_ignored=True)   # from RCSB
phi_psi_dict, ignored_res = phi_psi(local_pdb_file, return_ignored=True) # local PDB file

# ProDy testing

In [ ]:
# Label your protein
pro_filename = "md_0_1_trajectory_nowater.pdb"
pro_name = input("Enter a name label:")

from RamachanDraw import fetch, phi_psi, plot
plot(pro_filename)     # Draw the Ramachandran plot

In [ ]:
# prot = pd.parsePDB(PDB_id)    # from RCSB
pro = pd.parsePDB('md_0_1_trajectory_nowater.pdb', subset = 'calpha')   # local PDB file

# Print useful statistics
print("Radius of gyration:", pd.calcGyradius(pro))
print("Number of atoms:", pro.numAtoms())
print("Number of Coordinate sets:", pro.numCoordsets())
print("Number of residues:", pro.numResidues())
pd.showProtein(pro)

PRINCIPAL COMPONENT ANALYSIS

In [ ]:

pd.apps.prody_apps.prody_pca('md_0_1_trajectory_nowater.pdb', figformat='png', outall=True)

In [ ]:
# Prepare the ensemble
#pro = pd.parsePDB(PDB_id, subset='calpha')     # from RCSB
pro = pd.parsePDB('md_0_1_trajectory_nowater.pdb', subset = 'calpha')   # local PDB file

pro_selection = pro.select('resnum < ' + str(pro.numResidues()))
pro_ensemble = pd.Ensemble(pro_selection)
cs = pro_ensemble.getCoords() # get coordinate sets
print(cs)
pro_ensemble.addCoordset(cs)


pro_ensemble.setCoords(cs)

pro_ensemble.iterpose()

In [ ]:
# Run PCA 
pca = pd.PCA(pro_name)
pca.buildCovariance(pro_ensemble)
pca.calcModes()

In [ ]:
# Observe top 6 ranked principal components
for mode in pca[:6]:
    print(pd.calcFractVariance(mode).round(2))
# Save the principal modes
# pd.saveModel(pca)

ANISOTROPIC NETWORK MODEL


In [ ]:
anm = pd.ANM(pro_name) # instantiate ANM object
anm.buildHessian(pro_selection) # build Hessian matrix for selected atoms
anm.calcModes() # calculate normal modes
# saveModel(anm)

In [ ]:
# Access individual mode instances
slowest_mode = anm[0]
print( slowest_mode )
print( slowest_mode.getEigval().round(3) )

In [ ]:
# Observe top 6 ranked principal components
for mode in anm[:6]:
    print(pd.calcFractVariance(mode).round(2))

In [ ]:
# Confirm mode orthogonality - dot product of mode vectors
print((anm[0] * anm[1]).round(10))
print((anm[0] * anm[2]).round(10))

COMPARING EXPERIMENTAL AND THEORETICAL RESULTS

In [ ]:
# Compare overlap table of PCA and ANM
pd.printOverlapTable(pca[:6], anm[:6])
ot = pd.showOverlapTable(pca[:6], anm[:6])
# ot[2]
plt.savefig('overlap_table.png', bbox_inches='tight')

DATA OUTPUT

In [ ]:
# Write Normal Modes for PCA data
pd.writeNMD(pro_name + '_' + PDB_id + '_pca.nmd', pca[:6], pro_selection)         # NMD format for nm wizard
pd.writeArray('ubi_pca_modes.txt', pca.getArray(), format='%8.3f')     # text

In [ ]:
# Write Normal Modes for ANM data
pd.writeNMD(pro_name + '_' + PDB_id + '_anm.nmd', anm[:6], pro_selection)         # NMD format for nm wizard

In [ ]:
# pd.pathVMD('/Users/jacobnorth/Applications/VMD\ 1.9.4.app/Contents/MacOS/startup.command')
# pd.viewNMDinVMD('ubi_pca.nmd')

In [ ]:
pd.showSqFlucts(pca[::]);
pd.showSqFlucts(anm[::]);
pd.showNormedSqFlucts(anm[::]);
plt.savefig('sq_flucts.png', bbox_inches='tight')

STIFFNESS

In [ ]:
pro_hdr, header = pd.parsePDB(local_pdb_file, header=True)

# Define calphas 
calphas = pro.ca # define calphas
stiffness = pd.calcMechStiff(anm, calphas)  # calculate stiffnesses

# Illustrate mechanical stiffness of the protein
show = pd.showMechStiff(stiffness, calphas, cmap='jet_r')
plt.savefig('mech_stiffness.png', bbox_inches='tight')

# Show mean mechanical stiffness
show = pd.showMeanMechStiff(stiffness, calphas, header, 'A', cmap='jet_r')
plt.savefig('mean_mech_stiff.png', bbox_inches='tight')

# Calculate range of k, spring constant
pd.calcStiffnessRange(stiffness)

In [ ]:
# Pipe to VMD
# pd.writeVMDstiffness(stiffness, gfp, [3,7], [0,7.5], filename='1gfl_3-7aa')
# pd.writeVMDstiffness(stiffness, gfp, [3], [0,7], filename='1gfl_3')

CALCULATE DISTRIBUTION OF DEFORMATION

In [ ]:
d0 = pd.calcPairDeformationDist(anm, calphas, 3, 132)
show = pd.showPairDeformationDist(anm, calphas, 3, 132)
plt.savefig('pds_dist.png', bbox_inches='tight')

PRS MATRIX

In [ ]:
# Separate alpha carbons!
prot_ca = pd.parsePDB(local_pdb_file, subset='ca')
anm_prot = pd.ANM(pro_name)
anm_prot.buildHessian(prot_ca)
anm_prot.calcModes()

# Perturbative response
show = pd.showPerturbResponse(anm_prot, atoms=prot_ca)
plt.show()
plt.savefig('perturb_response.png', bbox_inches='tight')

# Save model
#pd.saveModel(anm_prot, pro_name, matrices=True)
#pd.writeNMD(pro_name+"_PRS", anm_prot, prot_ca)

EVOLUTIONARY ANALYSIS - EVOL

In [ ]:
pkey
type(pkey)
(list(pkey))[0]
print("PDB ID:", PDB_id)
print("full_MSA=", full_MSA)
print("MSA=", msa)

In [ ]:
# Download the full MSA file for protein family
pkey = pd.searchPfam(PDB_id).keys()      # obtain the key
full_MSA = pd.fetchPfamMSA((list(pkey))[0])        # Fetch the full MSA
msa = pd.parseMSA(full_MSA)        # Parse the MSA
# Refine MSA to remove gappy entries
msa_refine = pd.refineMSA(msa, label=PDB_id, rowocc=0.8, seqid=0.98)

In [ ]:
# Occupancy calculation
pd.showMSAOccupancy(msa_refine, occ='res')
calcMSAOccupancy(msa_refine, occ='res').min()   # Find the minimum

# Shannon entropy
entropy = calcShannonEntropy(msa_refine)
showShannonEntropy(entropy, indices)
mutinfo = buildMutinfoMatrix(msa_refine)
mutinfo_norm = applyMutinfoNorm(mutinfo, entropy, norm='minent')
mutinfo_corr = applyMutinfoCorr(mutinfo, corr='apc')
showMutinfoMatrix(mutinfo)
showMutinfoMatrix(mutinfo_corr, clim=[0, mutinfo_corr.max()], xlabel=pro_name)
writeArray(pro_name + 'array.txt', mutinfo)
# Sequence-structure comparison - http://prody.csb.pitt.edu/tutorials/evol_tutorial/comparison.html